<a href="https://colab.research.google.com/github/Cloud-Dark/tugas-1_komputasi_berbasis_jaringan/blob/main/__notebook__.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color = 'red'>
<h1>Analysis of Absenteeism<h1>
    
<hr>

# Introduction
* Feature Engineering Technics

<br>
<br>
<font color = 'blue'>
<b>Content: </b>

1. [Load Libraries](#1)
1. [Load Dataset](#2)
1. [Basic Data Analysis](#3)
1. [Feature Engineering Part-1](#4)
    * [Remove Constant Features](#5)
    * [Remove Quasi-Constant Features](#6)
    * [Remove Duplicate Features](#7)
    * [Remove Correlation Feature](#8)
    * [Remove Unnecessary Features](#9)   
1. [Observe Reason for Absence](#10)
1. [Feature Engineering Part-2](#11)
1. [Logistic Regression](#12)
    * [Standardize The Data](#13)
    * [Train-Test Split of Data](#14)
    * [Model](#15)
    * [Finding The Intercept & Coefficients](#16)
    * [Save The Model](#17)  


    
<hr>
   

<a id = "1"></a><br>
## Load Libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_selection import VarianceThreshold
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


<a id = "2"></a><br>
## Load Dataset

In [2]:
data = pd.read_csv("/kaggle/input/employee-absenteeism-prediction/Absenteeism-data.csv")
data.head()

FileNotFoundError: ignored

<a id = "3"></a><br>
## Basic Data Analysis

In [ ]:
data.describe()

,ID,Reason for Absence,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
count,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000
mean,17.951429,19.411429,222.347143,29.892857,36.417143,271.801774,26.737143,1.282857,1.021429,0.687143,6.761429
std,11.028144,8.356292,66.312960,14.804446,6.379083,40.021804,4.254701,0.668090,1.112215,1.166095,12.670082
min,1.000000,0.000000,118.000000,5.000000,27.000000,205.917000,19.000000,1.000000,0.000000,0.000000,0.000000
25%,9.000000,13.000000,179.000000,16.000000,31.000000,241.476000,24.000000,1.000000,0.000000,0.000000,2.000000
50%,18.000000,23.000000,225.000000,26.000000,37.000000,264.249000,25.000000,1.000000,1.000000,0.000000,3.000000
75%,28.000000,27.000000,260.000000,50.000000,40.000000,294.217000,31.000000,1.000000,2.000000,1.000000,8.000000
max,36.000000,28.000000,388.000000,52.000000,58.000000,378.884000,38.000000,4.000000,4.000000,8.000000,120.000000


In [ ]:
data.isnull().sum()

ID                           0
Reason for Absence           0
Date                         0
Transportation Expense       0
Distance to Work             0
Age                          0
Daily Work Load Average      0
Body Mass Index              0
Education                    0
Children                     0
Pets                         0
Absenteeism Time in Hours    0
dtype: int64

In [ ]:
# if you want to see all columns and raws:
# pd.options.display.max_columns = None
# pd.options.display.max_rows = None
# display(data)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         700 non-null    int64  
 1   Reason for Absence         700 non-null    int64  
 2   Date                       700 non-null    object 
 3   Transportation Expense     700 non-null    int64  
 4   Distance to Work           700 non-null    int64  
 5   Age                        700 non-null    int64  
 6   Daily Work Load Average    700 non-null    float64
 7   Body Mass Index            700 non-null    int64  
 8   Education                  700 non-null    int64  
 9   Children                   700 non-null    int64  
 10  Pets                       700 non-null    int64  
 11  Absenteeism Time in Hours  700 non-null    int64  
dtypes: float64(1), int64(10), object(1)
memory usage: 65.8+ KB


In [ ]:
data.shape

(700, 12)

<a id = "4"></a><br>
## Feature Engineering

<a id = "5"></a><br>
## Remove Constant Features

In [ ]:
# here for simplicity I will use only numerical variables
# select numerical columns:

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data_numerical = data[numerical_vars]
data_numerical.shape

(700, 11)

In [ ]:
# remove constant features
constant_features = [
    feat for feat in data_numerical.columns if data_numerical[feat].std() == 0
]

len(constant_features)

0

<a id = "6"></a><br>
## Remove Quasi-Constant Features
* There is no Quasi-Constant Feature

In [ ]:
# remove quasi-constant features
sel = VarianceThreshold(threshold=0.01) # 0.1 indicates 99% of observations approximately

sel.fit(data_numerical)  # fit finds the features with low variance

sum(sel.get_support()) # how many not quasi-constant?

11

<a id = "7"></a><br>
## Remove Duplicate Features

In [ ]:
data.duplicated().sum()

8

In [ ]:
data[data.duplicated()]

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
127,34,27,11/01/2016,118,10,37,308.593,28,1,0,0,2
128,34,27,12/01/2016,118,10,37,308.593,28,1,0,0,2
131,34,27,13/01/2016,118,10,37,308.593,28,1,0,0,2
305,5,23,19/10/2016,235,20,43,265.017,38,1,1,0,2
388,3,27,22/02/2017,179,51,38,251.818,31,1,0,0,3
389,3,27,24/02/2017,179,51,38,251.818,31,1,0,0,3
477,24,28,25/07/2017,246,25,41,230.290,23,1,0,0,2
496,24,28,05/09/2017,246,25,41,261.756,23,1,0,0,1


In [ ]:
data = data.drop_duplicates(keep="first").reset_index()

In [ ]:
data.shape

(692, 13)

In [ ]:
data.head()

,index,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2


<a id = "8"></a><br>
## Remove Correlation Feature
* Drop features if they have 80% correlation 

In [ ]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_features = correlation(data, 0.8)
len(set(corr_features))

0

<a id = "9"></a><br>
## Remove Unnecessary Features

In [ ]:
data.drop(["ID","index"],axis=1,inplace=True)
data.head()

,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,23,23/07/2015,289,36,33,239.554,30,1,2,1,2


<a id = "10"></a><br>
## Observe Reason for Absence

In [ ]:
# Reason for Absence
data["Reason for Absence"].unique()

array([26,  0, 23,  7, 22, 19,  1, 11, 14, 21, 10, 13, 28, 18, 25, 24,  6,
       27, 17,  8, 12,  5,  9, 15,  4,  3,  2, 16])

* 0: 'Unknown',
* 1: 'Certain infectious and parasitic diseases',
* 2: 'Neoplasms',
* 3: 'Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism',
* 4: 'Endocrine, nutritional and metabolic diseases',
* 5: 'Mental and behavioural disorders',
* 6: 'Diseases of the nervous system',
* 7: 'Diseases of the eye and adnexa',
* 8: 'Diseases of the ear and mastoid process',
* 9: 'Diseases of the circulatory system',
* 10: 'Diseases of the respiratory system',
* 11: 'Diseases of the digestive system',
* 12: 'Diseases of the skin and subcutaneous tissue',
* 13: 'Diseases of the musculoskeletal system and connective tissue',
* 14: 'Diseases of the genitourinary system',
* 15: 'Pregnancy, childbirth and the puerperium',
* 16: 'Certain conditions originating in the perinatal period',
* 17: 'Congenital malformations, deformations and chromosomal abnormalities',
* 18: 'Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified',
* 19: 'Injury, poisoning and certain other consequences of external causes',
* 20: 'External causes of morbidity and mortality',
* 21: 'Factors influencing health status and contact with health services',
* 22: 'Patient follow-up',
* 23: 'Medical consultation',
* 24: 'Blood donation',
* 25: 'Laboratory examination',
* 26: 'Unjustified absence',
* 27: 'Physiotherapy',
* 28: 'Dental consultation'

<a id = "11"></a><br>
## Feature Engineering Part-2

* One Hot Encoding

In [ ]:
reason_columns = pd.get_dummies(data["Reason for Absence"],drop_first=True)
reason_columns.head()

,1,2,3,4,5,6,7,8,9,10,...,18,19,21,22,23,24,25,26,27,28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
# Make Groups For Reasons
reason_type_1 = reason_columns.loc[:,:14].max(axis=1)
reason_type_2 = reason_columns.loc[:,15:17].max(axis=1)
reason_type_3 = reason_columns.loc[:,18:21].max(axis=1)
reason_type_4 = reason_columns.loc[:,22:].max(axis=1)

In [ ]:
# concat:
df = pd.concat([data.drop("Reason for Absence",axis=1), reason_type_1, reason_type_2, reason_type_3, reason_type_4],axis=1)
df.head()

,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,0,1,2,3
0,07/07/2015,289,36,33,239.554,30,1,2,1,4,0,0,0,1
1,14/07/2015,118,13,50,239.554,31,1,1,0,0,0,0,0,0
2,15/07/2015,179,51,38,239.554,31,1,0,0,2,0,0,0,1
3,16/07/2015,279,5,39,239.554,24,1,2,0,4,1,0,0,0
4,23/07/2015,289,36,33,239.554,30,1,2,1,2,0,0,0,1


In [ ]:
df.columns.values

array(['Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours', 0, 1, 2, 3],
      dtype=object)

In [ ]:
columns_names = ['Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours', "Reason_1", "Reason_2", "Reason_3", "Reason_4"]
df.columns = columns_names
df.head()

,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Reason_1,Reason_2,Reason_3,Reason_4
0,07/07/2015,289,36,33,239.554,30,1,2,1,4,0,0,0,1
1,14/07/2015,118,13,50,239.554,31,1,1,0,0,0,0,0,0
2,15/07/2015,179,51,38,239.554,31,1,0,0,2,0,0,0,1
3,16/07/2015,279,5,39,239.554,24,1,2,0,4,1,0,0,0
4,23/07/2015,289,36,33,239.554,30,1,2,1,2,0,0,0,1


In [ ]:
df.columns.values

array(['Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours', 'Reason_1',
       'Reason_2', 'Reason_3', 'Reason_4'], dtype=object)

In [ ]:
# reorder columns
column_names_reordered = ['Reason_1','Reason_2', 'Reason_3', 'Reason_4', 'Date', 'Transportation Expense', 'Distance to Work', 'Age',
                           'Daily Work Load Average', 'Body Mass Index', 'Education',
                           'Children', 'Pets', 'Absenteeism Time in Hours']
df = df[column_names_reordered]
df.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,0,0,0,1,23/07/2015,289,36,33,239.554,30,1,2,1,2


In [ ]:
# create a chechpoint: to reduce the risk of losing data
df_reason_mod = df.copy()

In [ ]:
# date:
# df_reason_mod["Date"].apply(lambda x: x.split("/"))
df_reason_mod["Date"] = pd.to_datetime(df_reason_mod["Date"], format="%d/%m/%Y")
df_reason_mod["Date"][0:5]

0   2015-07-07
1   2015-07-14
2   2015-07-15
3   2015-07-16
4   2015-07-23
Name: Date, dtype: datetime64[ns]

In [ ]:
# extract month
list_months = []
for i in range(len(df_reason_mod["Date"])):
    list_months.append(df_reason_mod["Date"][i].month)

In [ ]:
df_reason_mod["Month Value"] = list_months
df_reason_mod.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Month Value
0,0,0,0,1,2015-07-07,289,36,33,239.554,30,1,2,1,4,7
1,0,0,0,0,2015-07-14,118,13,50,239.554,31,1,1,0,0,7
2,0,0,0,1,2015-07-15,179,51,38,239.554,31,1,0,0,2,7
3,1,0,0,0,2015-07-16,279,5,39,239.554,24,1,2,0,4,7
4,0,0,0,1,2015-07-23,289,36,33,239.554,30,1,2,1,2,7


In [ ]:
# extract the day of the week:0,1,2,3,4,5,6
df_reason_mod["Date"][0].weekday()

1

In [ ]:
def day_to_weekday(date_value):
    return date_value.weekday()

In [ ]:
df_reason_mod["Day of the Week"] = df_reason_mod["Date"].apply(day_to_weekday)
df_reason_mod.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Month Value,Day of the Week
0,0,0,0,1,2015-07-07,289,36,33,239.554,30,1,2,1,4,7,1
1,0,0,0,0,2015-07-14,118,13,50,239.554,31,1,1,0,0,7,1
2,0,0,0,1,2015-07-15,179,51,38,239.554,31,1,0,0,2,7,2
3,1,0,0,0,2015-07-16,279,5,39,239.554,24,1,2,0,4,7,3
4,0,0,0,1,2015-07-23,289,36,33,239.554,30,1,2,1,2,7,3


## Education
* 1:High School
* 2:Graduate
* 3:Postgraduate
* 4:Master or Doctor

In [ ]:
# Education:
df_reason_mod["Education"].unique()

array([1, 3, 2, 4])

In [ ]:
df_reason_mod["Education"].value_counts()

1    575
3     73
2     40
4      4
Name: Education, dtype: int64

In [ ]:
# 1 => 0
# 2,3,4 => 1
df_reason_mod["Education"] = df_reason_mod["Education"].map({1:0,2:1,3:1,4:1})

In [ ]:
df_reason_mod["Education"].unique()

array([0, 1])

In [ ]:
df_reason_mod["Education"].value_counts()

0    575
1    117
Name: Education, dtype: int64

<a id = "12"></a><br>
## Logistic Regression

### Create Targets

In [ ]:
# create a checkpoint
df_model = df_reason_mod.copy()

In [ ]:
# use median cut-off hours and making targets 
df_model["Absenteeism Time in Hours"].median()

3.0

### Moderately Absent <= 3
### Excessively Absent > 4

In [ ]:
targets = np.where(df_model["Absenteeism Time in Hours"] > 3, 1, 0)
targets[0:10]

array([1, 0, 0, 1, 0, 0, 1, 1, 1, 1])

In [ ]:
# add to df
df_model["Excessive Absenteeism"] = targets
df_model.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Month Value,Day of the Week,Excessive Absenteeism
0,0,0,0,1,2015-07-07,289,36,33,239.554,30,0,2,1,4,7,1,1
1,0,0,0,0,2015-07-14,118,13,50,239.554,31,0,1,0,0,7,1,0
2,0,0,0,1,2015-07-15,179,51,38,239.554,31,0,0,0,2,7,2,0
3,1,0,0,0,2015-07-16,279,5,39,239.554,24,0,2,0,4,7,3,1
4,0,0,0,1,2015-07-23,289,36,33,239.554,30,0,2,1,2,7,3,0


In [ ]:
targets = pd.Series(targets)
targets.value_counts()

0    373
1    319
dtype: int64

In [ ]:
# drop Absenteeism Time in Hours
df_model.drop(["Absenteeism Time in Hours","Date"],axis=1,inplace=True)

In [ ]:
df_model is df_reason_mod

False

### Select Inputs

In [ ]:
unscaled_inputs = df_model.iloc[:,:-1]

<a id = "13"></a><br>
## Standardize The Data
* Omit Dummy Features

In [ ]:
# this class is just for selecting features to standardization:

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        self.scaler = StandardScaler(copy,with_mean,with_std)
        self.columns = columns
        self.mean_ = None
        self.var_ = None

    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self

    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]


In [ ]:
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4',
       'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Month Value', 'Day of the Week'], dtype=object)

In [ ]:
columns_to_scale = ['Month Value','Day of the Week', 'Transportation Expense', 'Distance to Work',
                    'Age', 'Daily Work Load Average', 'Body Mass Index', 'Children', 'Pets']

# Because these features are dummy variable
# columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4','Education'] 

In [ ]:
sc = CustomScaler(columns_to_scale)

In [ ]:
scaled_inputs = sc.fit_transform(unscaled_inputs)
scaled_inputs.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Month Value,Day of the Week
0,0,0,0,1,0.998885,0.409831,-0.529189,-0.803696,0.774506,0,0.869872,0.260694,0.175838,-0.685271
1,0,0,0,0,-1.582804,-1.147311,2.126691,-0.803696,1.010195,0,-0.028563,-0.594284,0.175838,-0.685271
2,0,0,0,1,-0.661851,1.425359,0.251952,-0.803696,1.010195,0,-0.926998,-0.594284,0.175838,-0.010723
3,1,0,0,0,0.847909,-1.688926,0.408180,-0.803696,-0.639631,0,0.869872,-0.594284,0.175838,0.663826
4,0,0,0,1,0.998885,0.409831,-0.529189,-0.803696,0.774506,0,0.869872,0.260694,0.175838,0.663826


<a id = "14"></a><br>
## Train-Test Split of Data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs,targets,train_size=0.8,shuffle=True,random_state=7)

In [ ]:
x_train.shape, y_train.shape

((553, 14), (553,))

In [ ]:
x_test.shape, y_test.shape

((139, 14), (139,))

<a id = "15"></a><br>
## Model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

### Training The Model

In [ ]:
reg = LogisticRegression()

In [ ]:
reg.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
reg.score(x_train,y_train)

0.7414104882459313

### Manually Check The Accuracy

In [ ]:
model_outputs = reg.predict(x_train)

In [ ]:
model_outputs == y_train

48      True
134     True
679     True
236    False
55      True
       ...  
579     True
502    False
537    False
196     True
175     True
Length: 553, dtype: bool

In [ ]:
np.sum(model_outputs == y_train)

410

In [ ]:
model_outputs.shape

(553,)

In [ ]:
np.sum(model_outputs == y_train) / model_outputs.shape[0]

0.7414104882459313

In [ ]:
np.sum(model_outputs == y_train) / model_outputs.shape[0] == reg.score(x_train,y_train)

True

<a id = "16"></a><br>
## Finding The Intercept & Coefficients

In [ ]:
reg.intercept_

array([-1.55208786])

In [ ]:
reg.coef_.T

array([[ 2.79011295],
       [ 1.0340849 ],
       [ 2.7373299 ],
       [ 0.82395998],
       [ 0.5624909 ],
       [ 0.0244116 ],
       [-0.12011658],
       [-0.06778212],
       [ 0.24480056],
       [-0.17750591],
       [ 0.47313108],
       [-0.30496399],
       [ 0.11536176],
       [-0.17782254]])

In [ ]:
# Make df to show better:
feature_name = unscaled_inputs.columns
summary_table = pd.DataFrame(data=feature_name,columns=["Feature Name"])
summary_table["Coefficient"] = reg.coef_.T
summary_table

,Feature Name,Coefficient
0,Reason_1,2.790113
1,Reason_2,1.034085
2,Reason_3,2.737330
3,Reason_4,0.823960
4,Transportation Expense,0.562491
5,Distance to Work,0.024412
6,Age,-0.120117
7,Daily Work Load Average,-0.067782
8,Body Mass Index,0.244801
9,Education,-0.177506


In [ ]:
# add intercept:
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ["Intercept", reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature Name,Coefficient
0,Intercept,-1.552088
1,Reason_1,2.790113
2,Reason_2,1.034085
3,Reason_3,2.737330
4,Reason_4,0.823960
5,Transportation Expense,0.562491
6,Distance to Work,0.024412
7,Age,-0.120117
8,Daily Work Load Average,-0.067782
9,Body Mass Index,0.244801


* Whichever weights/coefficient is bigger, its correspending feature is more important.

<hr> 

### Interpreting The Logistic Regression Coeff.

* log(odds) = intercept + b1x1 + b2x2 + b3x3 + .... + b14x14

In [ ]:
summary_table["Odds_ratio"] = np.exp(summary_table.Coefficient)
summary_table

,Feature Name,Coefficient,Odds_ratio
0,Intercept,-1.552088,0.211805
1,Reason_1,2.790113,16.282859
2,Reason_2,1.034085,2.812531
3,Reason_3,2.737330,15.445688
4,Reason_4,0.823960,2.279509
5,Transportation Expense,0.562491,1.755039
6,Distance to Work,0.024412,1.024712
7,Age,-0.120117,0.886817
8,Daily Work Load Average,-0.067782,0.934464
9,Body Mass Index,0.244801,1.277367


In [ ]:
summary_table.sort_values("Odds_ratio", ascending=False)

,Feature Name,Coefficient,Odds_ratio
1,Reason_1,2.790113,16.282859
3,Reason_3,2.737330,15.445688
2,Reason_2,1.034085,2.812531
4,Reason_4,0.823960,2.279509
5,Transportation Expense,0.562491,1.755039
11,Children,0.473131,1.605012
9,Body Mass Index,0.244801,1.277367
13,Month Value,0.115362,1.122279
6,Distance to Work,0.024412,1.024712
8,Daily Work Load Average,-0.067782,0.934464


### A feature is not particularly important:
* if its coeff is around 0
* if its odds ratio is around 1

#### So you can consider to drop these features: Education, Month Value, Distance to Work
<hr>

### Interpreting The Important Features

* Reason_1 : various diseases
* Reason_2 : pragnancy and giving birth
* Reason_3 : poising
* Reason_4 : light diseases

### Test The Model
* Often the test accuracy is 10-20% lower than the train accuracy (due to overfitting) 

In [ ]:
reg.score(x_test, y_test)

0.7410071942446043

In [ ]:
# predict_proba(x) : returns the probability estimates for all possible outputs
predict_proba = reg.predict_proba(x_test)
predict_proba

array([[0.70058726, 0.29941274],
       [0.29611565, 0.70388435],
       [0.50148666, 0.49851334],
       [0.281531  , 0.718469  ],
       [0.61671575, 0.38328425],
       [0.84372762, 0.15627238],
       [0.86687965, 0.13312035],
       [0.43392536, 0.56607464],
       [0.59026464, 0.40973536],
       [0.48498592, 0.51501408],
       [0.79879643, 0.20120357],
       [0.47688986, 0.52311014],
       [0.16185058, 0.83814942],
       [0.82585034, 0.17414966],
       [0.79310042, 0.20689958],
       [0.89407055, 0.10592945],
       [0.65005595, 0.34994405],
       [0.77120817, 0.22879183],
       [0.72441414, 0.27558586],
       [0.38718603, 0.61281397],
       [0.55962961, 0.44037039],
       [0.89407055, 0.10592945],
       [0.76414812, 0.23585188],
       [0.2334685 , 0.7665315 ],
       [0.14921487, 0.85078513],
       [0.62003794, 0.37996206],
       [0.74076341, 0.25923659],
       [0.87699765, 0.12300235],
       [0.29180124, 0.70819876],
       [0.76965496, 0.23034504],
       [0.

In [ ]:
# see sum is 1
0.72033435 + 0.27966565, 0.87854892 + 0.12145108

(1.0, 1.0)

In [ ]:
# 1. column: probality of being 0
# 2. column: probality of being 1
predict_proba.shape

(139, 2)

* if the probality is:
  * below 0.5, it places a 0
  * above 0.5, it places a 1

In [ ]:
predict_proba[:,1]

array([0.29941274, 0.70388435, 0.49851334, 0.718469  , 0.38328425,
       0.15627238, 0.13312035, 0.56607464, 0.40973536, 0.51501408,
       0.20120357, 0.52311014, 0.83814942, 0.17414966, 0.20689958,
       0.10592945, 0.34994405, 0.22879183, 0.27558586, 0.61281397,
       0.44037039, 0.10592945, 0.23585188, 0.7665315 , 0.85078513,
       0.37996206, 0.25923659, 0.12300235, 0.70819876, 0.23034504,
       0.10520317, 0.65383507, 0.37440467, 0.10353031, 0.20487377,
       0.10977079, 0.25308117, 0.37897135, 0.35308465, 0.34751207,
       0.79956432, 0.49633624, 0.2554706 , 0.86033955, 0.25217289,
       0.44103771, 0.24231802, 0.18566469, 0.30152275, 0.26656919,
       0.89917941, 0.27383614, 0.48222351, 0.87084979, 0.44536343,
       0.3363793 , 0.38235627, 0.72780149, 0.90239201, 0.81543169,
       0.59683891, 0.80389371, 0.16507998, 0.84774211, 0.26086682,
       0.89043798, 0.48203808, 0.67960384, 0.08541395, 0.17655621,
       0.29941274, 0.67579491, 0.51687357, 0.64936177, 0.52593

<a id = "17"></a><br>
## Save The Model

In [ ]:
import pickle

In [ ]:
# model: file name / wb: write bites / dump:save
with open("model", "wb") as file:
    pickle.dump(reg,file)

### Save The Scale

In [ ]:
with open("model_2", "wb") as file_2:
    pickle.dump(sc,file_2)

### See The Files

In [ ]:
from IPython.display import FileLink, FileLinks
FileLinks('.') #lists all downloadable files on server

./
  model
  model_2
  __notebook__.ipynb